---
title: Commonly used parameters for transpilation
description: Overview of commonly used parameters controlling quantum circuit transpilation in Qiskit.
---


# Commonly used parameters for transpilation

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>

This page describes some of the more commonly used parameters for local transpilation. These parameters are configured using arguments to [`generate_preset_pass_manager`](/docs/api/qiskit/qiskit.transpiler.generate_preset_pass_manager#qiskit.transpiler.generate_preset_pass_manager) or [`transpile`](/docs/api/qiskit/compiler#qiskit.compiler.transpile).

<span id="approx-degree"></span>
## Approximation degree

You can use the approximation degree to specify how closely you want the resultant circuit to match the desired (input) circuit.  This is a float in the range (0.0 - 1.0), where 0.0 is maximum approximation and 1.0 (default) is no approximation. Smaller values trade output accuracy for ease of execution (that is, fewer gates).  The default value is 1.0.

In two-qubit unitary synthesis (used in initial stages of all levels and for optimization stage with optimization level 3), this value specifies the target fidelity of the output decomposition. That is, how much error is introduced when a matrix representation of a circuit is converted to discrete gates. If the approximation degree is a lower value (more approximation), the output circuit from synthesis will differ more from the input matrix, but will also likely have fewer gates (because any arbitrary two-qubit operation can be decomposed perfectly with at most three CX gates) and is easier to run.

When the approximation degree is less than 1.0, circuits with one or two CX gates might be synthesized, leading to less error from the hardware, but more from the approximation. Since CX is the most expensive gate in terms of error, it might be beneficial to decrease the number of them at the cost of fidelity in synthesis (this technique was used to increase quantum volume on IBM&reg; devices: [Validating quantum computers using randomized model circuits](https://arxiv.org/abs/1811.12926)).

As an example, we generate a random two-qubit `UnitaryGate` which will be synthesized in the initial stage. Setting the `approximation_degree` less than 1.0 might generate an approximate circuit. We must also specify the `basis_gates` to let the synthesis method know which gates it can use for the approximate synthesis.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import UnitaryGate
from qiskit.quantum_info import random_unitary
from qiskit.transpiler import generate_preset_pass_manager

UU = random_unitary(4, seed=12345)
rand_U = UnitaryGate(UU)

qubits = QuantumRegister(2, name="q")
qc = QuantumCircuit(qubits)
qc.append(rand_U, qubits)
pass_manager = generate_preset_pass_manager(
    optimization_level=1,
    approximation_degree=0.85,
    basis_gates=["sx", "rz", "cx"],
)
approx_qc = pass_manager.run(qc)
print(approx_qc.count_ops()["cx"])

2


This yields an output of `2` because the approximation requires fewer CX gates.

<span id="seed"></span>
## Random number generator seed

Some parts of the transpiler are stochastic, so repeated transpilation runs may return different results. To obtain a reproducible result, you can set the seed for the pseudorandom number generator using the `seed_transpiler` argument. Repeated runs using the same seed will return the same results.

Example:

In [2]:
pass_manager = generate_preset_pass_manager(
    optimization_level=1, seed_transpiler=11, basis_gates=["sx", "rz", "cx"]
)
optimized_1 = pass_manager.run(qc)
optimized_1.draw("mpl")

<Image src="/docs/images/guides/common-parameters/extracted-outputs/dbc652e8-53a4-47a9-a66e-d9c1e5ef07c9-0.svg" alt="Output of the previous code cell" />

<span id="init-layout"></span>
## Initial layout

Before transpilation, the qubits contained in your circuit are virtual qubits that don't necessarily correspond to physical qubits on the target backend. You can specify the initial mapping of virtual qubits to physical qubits using the `initial_layout` argument. Note that the final qubit layout may differ from the initial layout because the transpiler may permute qubits using swap gates or other means.

In the example below, we construct an initial layout for the [`FakeSherbrooke`](/docs/api/qiskit-ibm-runtime/fake-provider-fake-sherbrooke#fakesherbrooke) mock backend by creating a [`Layout`](/docs/api/qiskit/qiskit.transpiler.Layout) object. Our layout maps the first qubit of our circuit to qubit 5 of Sherbrooke, and it maps the second qubit of our circuit to qubit 6 of Sherbrooke. Note that physical qubits are always represented by integers.

In [3]:
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit.transpiler import Layout

backend = FakeSherbrooke()

a, b = qubits
initial_layout = Layout({a: 5, b: 6})

pass_manager = generate_preset_pass_manager(
    optimization_level=1, backend=backend, initial_layout=initial_layout
)
transpiled_circ = pass_manager.run(qc)

transpiled_circ.draw("mpl", idle_wires=False)

<Image src="/docs/images/guides/common-parameters/extracted-outputs/e18c034c-eb26-4d9d-81d7-37e0eafa17c7-0.svg" alt="Output of the previous code cell" />

In addition to specifying a Layout object, you can also pass a list of integers, where the $i$-th element of the list contains the physical qubit that the $i$-th qubit should be mapped to. For example:

In [4]:
initial_layout = [5, 6]

pass_manager = generate_preset_pass_manager(
    optimization_level=1, backend=backend, initial_layout=initial_layout
)
transpiled_circ = pass_manager.run(qc)

transpiled_circ.draw("mpl", idle_wires=False)

<Image src="/docs/images/guides/common-parameters/extracted-outputs/a7800d8a-7354-48e4-a55f-f902ae28c875-0.svg" alt="Output of the previous code cell" />

You can use the [`plot_error_map`](/docs/api/qiskit/qiskit.visualization.plot_error_map) function to generate a diagram of the device graph with error information and with the physical qubits labeled. You can also view similar diagrams on the [Compute resources](https://quantum.cloud.ibm.com/computers) page.

In [5]:
from qiskit.visualization import plot_error_map

plot_error_map(backend, figsize=(30, 24))

<Image src="/docs/images/guides/common-parameters/extracted-outputs/8df57c6a-1ff4-4d58-9b7e-4378452c3025-0.svg" alt="Output of the previous code cell" />

<span id="xx-method"></span>
## Transpiler stage and plugin options

These options are suffixed with `_method`. They influence how the transpiler works and are used to try and get better, different, or specific output from the transpiler.

* `init_method` (str) - The plugin to use for the initialization stage.

* `layout_method` (str) - The layout selection pass (`trivial`, `dense`, `sabre`). This can also be the external plugin name to use for the layout stage.

* `optimization_method` (str) - The plugin to use for the optimization stage.

* `routing_method` (str) - Name of routing pass (`basic`, `lookahead`, `default`, `sabre`, `none`). This can also be the external plugin name to use for the routing stage.

* `scheduling_method` (str) - Name of scheduling pass. This can also be the external plugin name to use for the scheduling stage.
  * `as_soon_as_possible`: Schedule instructions greedily: as early as possible on a qubit resource (alias: `asap`).
  * `as_late_as_possible`: Schedule instructions late.  That is, keep qubits in the ground state when possible (alias: `alap`).

* `translation_method` (str) - Name of translation pass (`unroller`, `translator`, `synthesis`). This can also be the external plugin name to use for the translation stage.

* `unitary_synthesis_method` (str) - The name of the unitary synthesis method to use. By default `default` is used.

<Admonition>
  To see a list of all installed plugins for a given stage, run [`list_stage_plugins("stage_name")`](/docs/api/qiskit/transpiler_plugins). For example, if you want to see a list of all installed plugins for the routing stage, run `list_stage_plugins(routing)`.
</Admonition>

## Next steps

<Admonition type="tip" title="Recommendation">
    - Review the [Default options and configuration settings](defaults-and-configuration-options) topic.
    - Learn how to [Set the optimization level.](set-optimization)
    - Try the [Compare transpiler settings](/docs/guides/circuit-transpilation-settings) guide.
    - Review the [transpiler API documentation.](/docs/api/qiskit/transpiler)

</Admonition>